In [ ]:
import pandas as pd
import pickle
import ast

from tqdm import tqdm
import os
import random
import re
import warnings

os.getcwd()
tqdm.pandas()
warnings.filterwarnings('ignore')

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp, Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

match = pd.read_csv("/data/log-data-2024/20241127_Final/Meta_20241127_optimized.csv")
match = dict(zip(match["optimized_new_contents_id"], match["contents_id"]))

prediction = pd.read_csv("/data/log-data-2024/20241127_Final/8man_sample_20241127_predicted_treatment2.csv")
prediction["treatment2 prediction"] = prediction["treatment2 prediction"].apply(lambda x : ast.literal_eval(x))

df = pd.read_csv("/data/log-data-2024/2.sequence_generate_ksc/data/sequence_device_match_241127.csv")
cf = pd.read_csv("/data/log-data-2024/20241127_Final/input_search_final_20241127.txt", sep = "\t", header = None)
sequence = pd.concat([df, cf], axis = 1)
sequence = dict(zip(sequence["device_id"], sequence[0]))
prediction["sequence"] = [sequence[i].split()[1:] for i in prediction["treatment2"]]

with open(file='/data/log-data-2024/20241127_Final/match_dict_final2.pickle', mode='rb') as f:
    match_dict = pickle.load(f)
rev = dict(zip(match_dict.values(), match_dict.keys()))

with open(file='/data/log-data-2024/20241127_Final/content_meta_dict_20241127.pickle', mode='rb') as f:
    meta = pickle.load(f)

temp = []
for i in tqdm(prediction["treatment2 prediction"]):
    ttemp = {}
    for w in i:
        try:
            ttemp[int(w)] = meta[int(w)]
        except:
            pass
    temp.append(ttemp)
prediction["treatment2 prediction meta"] = temp

temp = []
for i in tqdm(prediction["sequence"]):
    ttemp = {}
    for w in i:
        try:
            ttemp[int(w)] = meta[int(w)]
        except:
            pass
    temp.append(ttemp)
    
prediction["treatment2 sequence meta"] = temp

sequence = prediction.copy()
sequence

In [ ]:
z = re.compile("genre_info: \[.+\], actor_info")
genres = []
for i in meta:
    try:
        temp = z.findall(meta[i])[0][13:-13].split(", ")
        for w in temp:
            genres.append(w)
    except:
        pass
genres = list(set(genres))
genre_list = []
for i in genres:
    genre_list.append(i)
    
genre_list.remove("kids")

In [ ]:
os.environ["OLLAMA_LLM_LIBRARY"] = "cpu_avx2"

# model_name = 'llama3.1:70b'
model_name = 'llama3.1'

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = Ollama(
    model=model_name,
    temperature=0.75,
    verbose=False
)

In [ ]:
from collections import Counter

gl = []
for i in sequence["treatment2 prediction meta"]:
    gen = []
    for w in i:
        try:
            for p in z.findall(i[w])[0][13:-13].split(", "):
                gen.append(p)
        except:
            pass
    length = len(gen)
    gen = dict(sorted(Counter(gen).items(), key=lambda x: x[1], reverse=True))

    genre_list_temp = []
    ratio = 0
    for i in gen:
        if (ratio < 0.4) and (gen[i] > 3):
            ratio += gen[i]/length
            genre_list_temp.append(i)
        else:
            break
    try:
        genre_list_temp.remove("kids")
    except:
        pass
    gl.append(genre_list_temp)
            
sequence["major_genre"] = gl

In [ ]:
nt = re.compile("genre_info: None")

result = []

for num in tqdm(sequence.index):

    front_order = []
    back_order = []
    for i in sequence["treatment2 prediction"].loc[num]:
        m = sequence["treatment2 prediction meta"].loc[num]
        gl = sequence["major_genre"].loc[num]
        
        try:
            meta_temp = m[i]

            try:
                meta_temp = z.findall(meta[i])[0][13:-13].split(", ")
            except:
                meta_temp = [nt.findall(meta_temp)[0][12:]]

            if any(element in meta_temp for element in gl):
                front_order.append(i)
            else:
                back_order.append(i)

            front_back = front_order + back_order
        except:
            pass
    result.append(front_back)
    
sequence["treatment2 prediction"] = result

In [ ]:
temp1 = []
temp2 = []
for i in tqdm(sequence["treatment2 prediction"]):
    ttemp1 = []
    ttemp2 = []
    for w in i:
        try:
            ttemp1.append(match_dict[str(w)])
            ttemp2.append(match[int(w)])
        except:
            pass
    temp1.append(ttemp1)
    temp2.append(ttemp2)
    
sequence["treatment2 prediction name"] = temp1
sequence["treatment2 prediction set_id"] = temp2

temp1 = []
temp2 = []
for i in tqdm(sequence["sequence"]):
    ttemp1 = []
    ttemp2 = []
    for w in i:
        try:
            ttemp1.append(match_dict[str(w)])
            ttemp2.append(match[int(w)])
        except:
            pass
    temp1.append(ttemp1)
    temp2.append(ttemp2)
    
sequence["treatment2 sequence name"] = temp1
sequence["treatment2 sequence set_id"] = temp2

In [ ]:
# sequence.to_csv("/data/log-data-2024/20241127_Final/8man_sample_20241127_treatment2_final.csv")
sequence = pd.read_csv("/data/log-data-2024/20241127_Final/8man_sample_20241127_treatment2_final.csv")
sequence = sequence[["treatment2", "treatment2 sequence name", "treatment2 prediction name", "treatment2 sequence meta", "major_genre"]]
sequence.columns = ["device_id", "sequence", "prediction", "meta", "major_genre"]

In [ ]:
bad_examples = ["Top dramas for you", "We recommend exciting adventure contents!", "Thrill-seekers likes to see our latest picks"]

def generate_message(table):
    res = []
    
    system = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    Your are a smart TV recommendation message generator.
    <|eot_id|>
    '''
    
    user = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    Generate a creative recommendation message to a customer who has previously watched {genre}.
    Do not specify the name of the contents or the name of the genre.
    Instead, define the characteristics of the customer with the given genres.
    The message should be a sentence consists of maximum 8 words.
    Simply output one recommendation message only.
    
    Here's some examples we prefer:
    ["Drama lovers may also like our latest picks.",
    "Try contents that are admired by humor seekers.",
    "You might find yourself on the edge of your seat with this gripping, spine-tingling experience."]
    
    Here's some bad examples we do not prefer:
    {bad_examples}
    <|eot_id|>
    '''
    
    for i in tqdm(table.index):

        template_text = system + user

        prompt = PromptTemplate.from_template(template_text)

        formatted_inputs = {
            "genre" : table["major_genre"].loc[i],
            "bad_examples" : bad_examples
        }

        chain = prompt | llm | StrOutputParser()
        response = chain.invoke(formatted_inputs)
        
        res.append(response)
        
    res = pd.DataFrame({"device_id" : table["device_id"], "message" : res, "sequence" : table["sequence"],
                        "prediction" : table["prediction"], "meta" : table["meta"],
                        "major_genre" : table["major_genre"]})
    
    return res

def approve_message(table):
    
    system = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    You are a marketing manager.
    <|eot_id|>
    '''
    user = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    Evaluate whether the message meets the conditions below.
    Output "Approved" if the message meets all conditions below, otherwise "Rejected".
    Output the basis of the decision with the decision per condition.
    Message: {msg}
    Condition 1. The message should not recommend specific contents or genres.
    Condition 2. Recommending contents to specific type of customers is allowed.
    Condition 3: The message should be grammatically correct and fluent.
    Do not check other conditions than listed.
    Genres are listed in {grs}
    <|eot_id|>
    '''

    template_text = system + user

    prompt = PromptTemplate.from_template(template_text)

    filtering = []
    for i in table.index:
        classify = 1
        target = table["message"].loc[i]
        comp = table["major_genre"].loc[i]
        if ("Thrill" in target) & ("thriller" not in comp):
            classify = 0
        elif ("thrill" in target) & ("thriller" not in comp):
            classify = 0
        else:
            compare = list(set(genre_list) - set(comp))

            for w in compare:
                if w in target:
                    classify = 0
                    break
                else:
                    pass
        if classify == 1:
            filtering.append(1)
        else:
            filtering.append(0)
            
    result = []
    for a, i in tqdm(enumerate(table.index)):
        if filtering[a] == 0:
            result.append(0)
        else:
            formatted_inputs = {
                "msg" : target,
                "grs" : genres,
                "meta": table["meta"].loc[i]
            }

            chain = prompt | llm | StrOutputParser()
            response = chain.invoke(formatted_inputs)

            result.append(response)
            
    table["class"] = result
    table["class"] = [0 if i == 0 else 0 if "Rejected" in i else 0 if (("Rejected" not in i) & ("Approved" not in i)) else 1 for i in table["class"]]
    
    return table

In [ ]:
result = pd.DataFrame(columns = ["device_id", "message", "sequence", "prediction", "meta", "class"])
seq = sequence.copy()
df = seq.copy()

while len(result.index) < len(seq.index) * (2/5):
    df = generate_message(df)
    df = approve_message(df)
    
    approved = df[df["class"] == 1]
    df = df[df["class"] == 0]
    
    bad_examples = list(df["message"].loc[:10])
    result = pd.concat([result, approved], axis = 0).reset_index(drop = True)
    
    result.to_csv("20241127_llmemb_treatment.csv")
    df.to_csv("20241127_llmemb_control.csv")